In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import os
import numpy as np
import pandas as pd
from shapely.geometry import Point, Polygon

In [4]:
postcode=gpd.read_file(r"D:\RISHIN\01_50m\combined_shapefile\USWF_Mapping_Task\US_v24_Admin_Boundaries\US_Boundaries.gdb\a0000000b.gdbtable")

In [5]:
postcode=postcode.rename(columns={'GDMID':'POSTCODE'})

In [6]:

# Create a new geometry column based on Longitude and Latitude columns
postcode['geometry'] = postcode.apply(lambda row: Point(row['LONGITUDE'], row['LATITUDE']), axis=1)

# Select the relevant columns (Longitude, Latitude, POST, and geometry)
postcode_point= postcode[[ 'POSTCODE', 'geometry']]


In [7]:
admin1=gpd.read_file(r"D:\RISHIN\01_50m\combined_shapefile\USWF_Mapping_Task\US_v24_Admin_Boundaries\US_Boundaries.gdb\a00000009.gdbtable")

In [8]:
admin1=admin1[admin1['NAME']!="Alaska"]


In [9]:
admin1.rename(columns={'GDMID':'Admin_1'},inplace=True)

In [10]:
admin1=admin1[['Admin_1','geometry']]

In [11]:
postcode2=postcode[['POST','geometry']]

In [12]:
admin2=gpd.read_file(r"D:\RISHIN\01_50m\combined_shapefile\USWF_Mapping_Task\US_v24_Admin_Boundaries\US_Boundaries.gdb\a0000000a.gdbtable")

In [13]:
admin2.rename(columns={'GDMID':'Admin_2'},inplace=True)

In [14]:
postcode_admin1 = gpd.sjoin(postcode_point, admin1,how="left" , predicate="intersects")


In [15]:
admin2=admin2[['Admin_2','geometry']]

In [16]:
postcode_admin1.drop(columns=['index_right'],inplace=True)

In [17]:
postadmincomb=gpd.sjoin(postcode_admin1,admin2,how="left",predicate="intersects")

In [18]:
postadmincomb.drop(columns={"index_right"})

,POSTCODE,geometry,Admin_1,Admin_2
0,9.999940e+10,POINT (-158.10859 59.90543),NaN,9.999920e+10
1,9.999940e+10,POINT (-151.03538 66.00359),NaN,9.999920e+10
2,9.999940e+10,POINT (-156.98596 57.5165),NaN,9.999920e+10
3,9.999940e+10,POINT (-153.19999 60.1854),NaN,9.999920e+10
4,9.999940e+10,POINT (-154.59103 60.56418),NaN,9.999920e+10
...,...,...,...,...
32078,9.999950e+10,POINT (-132.95814 55.55475),NaN,9.999920e+10
32079,9.999950e+10,POINT (-131.49473 55.12774),NaN,9.999920e+10
32080,9.999950e+10,POINT (-133.35628 56.23458),NaN,9.999920e+10
32081,9.999950e+10,POINT (-132.37887 56.45099),NaN,9.999920e+10


In [19]:
postadmincomb2=postadmincomb.drop(columns={"geometry"})

In [20]:
postcode=gpd.read_file(r"D:\RISHIN\01_50m\combined_shapefile\USWF_Mapping_Task\US_v24_Admin_Boundaries\US_Boundaries.gdb\a0000000b.gdbtable")

In [21]:
postcode=postcode.rename(columns={'GDMID':'POSTCODE'})

In [22]:
postcode2=postcode[["POSTCODE",'geometry']]

In [23]:
lookup_file = postcode2.merge(postadmincomb2, on='POSTCODE', how='left')


In [24]:
lookup_file.drop(columns=['index_right'],inplace=True)

In [25]:
lookup_file =lookup_file.dropna(subset=['Admin_1'])


In [26]:
lookup_file

,POSTCODE,geometry,Admin_1,Admin_2
11,9.999940e+10,"MULTIPOLYGON (((-121.04557 40.25647, -121.0454...",99999106.0,9.999921e+10
12,9.999940e+10,"MULTIPOLYGON (((-171.74147 25.76047, -171.7414...",99999115.0,9.999922e+10
13,9.999940e+10,"MULTIPOLYGON (((-113.33411 32.1866, -113.33411...",99999104.0,9.999920e+10
14,9.999940e+10,"MULTIPOLYGON (((-110.72622 32.35582, -110.7262...",99999104.0,9.999920e+10
15,9.999940e+10,"MULTIPOLYGON (((-118.48421 35.55811, -118.4841...",99999106.0,9.999921e+10
...,...,...,...,...
31871,9.999950e+10,"MULTIPOLYGON (((-118.90976 46.08625, -118.9090...",99999153.0,9.999925e+10
31872,9.999950e+10,"MULTIPOLYGON (((-118.5439 46.78039, -118.54333...",99999153.0,9.999925e+10
31873,9.999950e+10,"MULTIPOLYGON (((-117.38262 45.99814, -117.3825...",99999153.0,9.999925e+10
31874,9.999950e+10,"MULTIPOLYGON (((-117.37766 46.2082, -117.37759...",99999153.0,9.999925e+10


In [28]:
lookup_file.to_file(r"D:\RISHIN\01_50m\lookup_file\lookup_file.gpkg",driver='GPKG')

Batch processing now

In [29]:
# Paths for input file, lookup shapefile, and output folder
input_txt_file = r"D:\RISHIN\01_50m\combined_shapefile\USWF_Mapping_Task\File_Name.txt"

lookup_shapefile= r"D:\RISHIN\01_50m\lookup_file\lookup_file.gpkg"
output_folder = r"D:\RISHIN\01_50m\out_csv_new"

In [30]:
import os
import geopandas as gpd
from shapely.geometry import Polygon, Point
from concurrent.futures import ThreadPoolExecutor, as_completed

# Define the main function to process each file
def batch_process(input_txt_file, lookup_shapefile, output_folder):
    # Load the lookup shapefile
    lookup_gdf = gpd.read_file(lookup_shapefile).to_crs("EPSG:4326")
    lookup_gdf = lookup_gdf[['POSTCODE','geometry', 'Admin_2', 'Admin_1'] + [col for col in lookup_gdf.columns if col not in ['POSTCODE','geometry' , 'Admin_2', 'Admin_1']]]
    # Convert POSTCODE, Admin_1, and Admin_2 to integers (removing the decimal .0)
    for column in ['POSTCODE', 'Admin_2', 'Admin_1']:
        lookup_gdf[column] = lookup_gdf[column].astype('Int64')  # Use 'Int64' to allow nulls if any
   
    # Read the list of filenames from the input text file
    with open(input_txt_file, 'r') as f:
        file_names = f.read().splitlines()

    # Ensure output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Use ThreadPoolExecutor for multithreading
    with ThreadPoolExecutor() as executor:
        # Submit tasks to the thread pool
        futures = {executor.submit(process_single_file, file_name, lookup_gdf, output_folder): file_name for file_name in file_names}
        
        # Process each completed task
        for future in as_completed(futures):
            file_name = futures[future]
            try:
                future.result()
                print(f"Processed and saved: {file_name}_lookup.csv")
            except Exception as e:
                print(f"Error processing {file_name}: {e}")

# Function to process a single file
def process_single_file(file_name, lookup_gdf, output_folder):
    long, lat = process_file_name(file_name)
    
    # Create the polygon grid and centroids from the lower-right corner
    polygon_gdf, centroid_gdf = create_polygon_grid(long, lat)
    
    # Perform the spatial join to find which centroids intersect the lookup shapefile
    joined_gdf = gpd.sjoin(centroid_gdf, lookup_gdf, how="left", predicate="intersects")
    
    # Extract Longitude and Latitude from the geometry column
    joined_gdf["Longitude"] = joined_gdf.geometry.x
    joined_gdf["Latitude"] = joined_gdf.geometry.y
    joined_gdf = joined_gdf.drop(columns={"geometry", "index_right"})
    
    # Reorder columns so 'Longitude' and 'Latitude' are the first two columns
    joined_gdf = joined_gdf[['Longitude', 'Latitude'] + [col for col in joined_gdf.columns if col not in ['Longitude', 'Latitude']]]
    
    # Define the output CSV file path with filename suffix
    output_csv_file = os.path.join(output_folder, f"{file_name}_lookup.csv")
    
    # Save the joined result as a CSV file
    joined_gdf.to_csv(output_csv_file, index=False)

# Function to process the file name and obtain the longitude and latitude
def process_file_name(file_name):
    parts = file_name.split("_")
    third_word = parts[2]
    first_num = int(third_word[0])
    long = int(third_word[1:3])
    lat = int(third_word[3:5])
    
    # Ensure the longitude is negative for the USA region
    if first_num == 3:
        return -long, lat
    elif first_num == 4:
        return -100 - long, lat
    else:
        raise ValueError("Unexpected first number in the third word")

# Function to create a polygon grid in a 100 km² area with 50 m spacing, starting from lower-right corner
def create_polygon_grid(lower_right_long, lower_right_lat, grid_size=1, spacing=0.0005):
    num_points = int(grid_size / spacing)
    polygons = []
    centroids = []

    for i in range(num_points):
        for j in range(num_points):
            lr_long = round(lower_right_long - (i * spacing), 5)
            lr_lat = round(lower_right_lat + (j * spacing), 5)
            ul_long = round(lr_long - spacing, 5)
            ul_lat = round(lr_lat + spacing, 5)
            
            polygon = Polygon([
                (lr_long, lr_lat), (lr_long, ul_lat),
                (ul_long, ul_lat), (ul_long, lr_lat),
                (lr_long, lr_lat)
            ])
            polygons.append(polygon)
            
            centroid_long = round(lr_long - (spacing / 2), 5)
            centroid_lat = round(lr_lat + (spacing / 2), 5)
            centroid = Point(centroid_long, centroid_lat)
            centroids.append(centroid)

    polygon_gdf = gpd.GeoDataFrame(geometry=polygons, crs="EPSG:4326")
    centroid_gdf = gpd.GeoDataFrame(geometry=centroids, crs="EPSG:4326")

    return polygon_gdf, centroid_gdf


In [31]:
batch_process(input_txt_file, lookup_shapefile, output_folder)


Processed and saved: FUEL_US_36844.DAT_lookup.csv
Processed and saved: FUEL_US_36747.DAT_lookup.csv
Processed and saved: FUEL_US_36748.DAT_lookup.csv
Processed and saved: FUEL_US_36647.DAT_lookup.csv
Processed and saved: FUEL_US_36746.DAT_lookup.csv
Processed and saved: FUEL_US_36744.DAT_lookup.csv
Processed and saved: FUEL_US_36745.DAT_lookup.csv
Processed and saved: FUEL_US_36843.DAT_lookup.csv
Processed and saved: FUEL_US_36545.DAT_lookup.csv
Processed and saved: FUEL_US_36645.DAT_lookup.csv
Processed and saved: FUEL_US_36644.DAT_lookup.csv
Processed and saved: FUEL_US_36646.DAT_lookup.csv
Processed and saved: FUEL_US_36848.DAT_lookup.csv
Processed and saved: FUEL_US_36847.DAT_lookup.csv
Processed and saved: FUEL_US_36845.DAT_lookup.csv
Processed and saved: FUEL_US_36846.DAT_lookup.csv
Processed and saved: FUEL_US_36941.DAT_lookup.csv
Processed and saved: FUEL_US_36942.DAT_lookup.csv
Processed and saved: FUEL_US_36943.DAT_lookup.csv
Processed and saved: FUEL_US_36947.DAT_lookup.csv


In [32]:
"""# Define the main function to process each file
def batch_process(input_txt_file, lookup_shapefile, output_folder):
    # Load the lookup shapefile
    lookup_gdf = gpd.read_file(lookup_shapefile).to_crs("EPSG:4326")
    #lookup_gdf=lookup_gdf.drop(columns={"index_right"})
    
    # Read the list of filenames from the input text file
    with open(input_txt_file, 'r') as f:
        file_names = f.read().splitlines()

    for file_name in file_names:
        try:
            # Process the filename to extract longitude and latitude
            long, lat = process_file_name(file_name)
            
            # Create the polygon grid and centroids from the lower-right corner
            polygon_gdf, centroid_gdf = create_polygon_grid(long, lat)
            
            # Perform the spatial join to find which centroids intersect the lookup shapefile
            joined_gdf = gpd.sjoin(centroid_gdf, lookup_gdf, how="left", predicate="intersects")
            # Extract Longitude and Latitude from the geometry column
            joined_gdf["Longitude"] = joined_gdf.geometry.x
            joined_gdf["Latitude"] = joined_gdf.geometry.y
            joined_gdf=joined_gdf.drop(columns={"geometry","index_right"})
            # Reorder columns so 'Longitude' and 'Latitude' are the first two columns
            joined_gdf = joined_gdf[['Longitude', 'Latitude'] + [col for col in joined_gdf.columns if col not in ['Longitude', 'Latitude']]]

        
            
            # Define the output CSV file path with filename suffix
            output_csv_file = os.path.join(output_folder, f"{file_name}_lookup.csv")
            
            # Save the joined result as a CSV file
            joined_gdf.to_csv(output_csv_file, index=False)
            print(f"Processed and saved: {output_csv_file}")
        
        except Exception as e:
            print(f"Error processing {file_name}: {e}")

# Function to process the file name and obtain the longitude and latitude
def process_file_name(file_name):
    parts = file_name.split("_")
    third_word = parts[2]
    first_num = int(third_word[0])
    long = int(third_word[1:3])
    lat = int(third_word[3:5])
    
    # Ensure the longitude is negative for the USA region
    if first_num == 3:
        return -long, lat
    elif first_num == 4:
        return -100 - long, lat
    else:
        raise ValueError("Unexpected first number in the third word")

# Function to create a polygon grid in a 100 km² area with 50 m spacing, starting from lower-right corner size 1 dgre=100km
def create_polygon_grid(lower_right_long, lower_right_lat, grid_size=1, spacing=0.0005):
    num_points = int(grid_size / spacing)
    polygons = []
    centroids = []

    for i in range(num_points):
        for j in range(num_points):
            lr_long = round(lower_right_long - (i * spacing),5)
            lr_lat = round(lower_right_lat + (j * spacing),5)
            ul_long = round(lr_long - spacing,5)
            ul_lat = round(lr_lat + spacing,5)
            
            polygon = Polygon([
                (lr_long, lr_lat), (lr_long, ul_lat),
                (ul_long, ul_lat), (ul_long, lr_lat),
                (lr_long, lr_lat)
            ])
            polygons.append(polygon)
            
            centroid_long = round(lr_long - (spacing / 2),5)
            centroid_lat = round(lr_lat + (spacing / 2),5)
            centroid = Point(centroid_long, centroid_lat)
            centroids.append(centroid)

    polygon_gdf = gpd.GeoDataFrame(geometry=polygons, crs="EPSG:4326")
    centroid_gdf = gpd.GeoDataFrame(geometry=centroids, crs="EPSG:4326")

    return polygon_gdf, centroid_gdf


# Ensure output folder exists
os.makedirs(output_folder, exist_ok=True)"""



'# Define the main function to process each file\ndef batch_process(input_txt_file, lookup_shapefile, output_folder):\n    # Load the lookup shapefile\n    lookup_gdf = gpd.read_file(lookup_shapefile).to_crs("EPSG:4326")\n    #lookup_gdf=lookup_gdf.drop(columns={"index_right"})\n    \n    # Read the list of filenames from the input text file\n    with open(input_txt_file, \'r\') as f:\n        file_names = f.read().splitlines()\n\n    for file_name in file_names:\n        try:\n            # Process the filename to extract longitude and latitude\n            long, lat = process_file_name(file_name)\n            \n            # Create the polygon grid and centroids from the lower-right corner\n            polygon_gdf, centroid_gdf = create_polygon_grid(long, lat)\n            \n            # Perform the spatial join to find which centroids intersect the lookup shapefile\n            joined_gdf = gpd.sjoin(centroid_gdf, lookup_gdf, how="left", predicate="intersects")\n            # Extra